In [ ]:
display_expanded_search()

In [ ]:
get_in_out_and_twoway_conns()
add_geospatial_info()
add_network_context()

In [ ]:
print anchor_ip + ": connected to " + str(len(inbound.keys())) + " IPs"
top_inbound_b = get_top_bytes(inbound,50)

In [ ]:
top_inbound_conns = get_top_conns(inbound,50)
top_inbound_b.update(top_inbound_conns) # merge the two dictionaries

**Threat Summary**

In [ ]:
display_threat_box(anchor_ip)

<h4> Save for Storyboard </h4>

In [ ]:
generate_attack_map_file(anchor_ip, top_inbound_b, outbound, twoway)
generate_stats(anchor_ip, top_inbound_b, outbound, twoway, threat_name)
generate_dendro(anchor_ip, top_inbound_b, outbound, twoway, t_date)
details_inbound(anchor_ip,top_inbound_b)

In [ ]:
import struct, socket
import numpy as np
import linecache, bisect
import csv
import operator
import json
import os
import ipywidgets as widgets # if your using jupyter/ipython < 1.4 please comment this line and uncomment the lines below
#from IPython.html import widgets
#from IPython.html.widgets import interact, interactive, fixed
from IPython.display import display

## The environment Variables in duxbay.conf are nedeed to complete the paths in the below variables please be sure that
## they are loaded correctly
!source /etc/duxbay.conf
!export LUSER

contxtpath =  os.environ['LUSER'] + '/ipython/iploc/'
iploc = contxtpath + 'iploc.csv'
nwloc = contxtpath + 'networkcontext_1.csv'

## Please replace <date> with the date of the data day in this format YYYYMMDD
t_date = '<date>'
spath =  os.environ['LUSER'] + '/ipython/user/'+ t_date + '/'
sconnect = spath + 'flow_scores.csv'
sconnectbu = spath + 'flow_scores_bu.csv'
threats_file = spath + 'threats.csv'
anchor_ip = ''
ir_f = ''
threat_name = ''
# load IP location information -- LONG step
# load fromIP into ndarray
iplist = np.loadtxt(iploc,dtype=np.uint32,delimiter=',',usecols={0},
                    converters={0: lambda s: np.uint32(s.replace('"',''))})

**Functions Definition**

In [ ]:
def display_controls(ip_list):
    container = widgets.HBox(width=550, height=150)    
    h_container = widgets.Box(width=550, height=200)    
    label = widgets.HTML(value='<h2>Expanded Search</h2>')
    ip_select = widgets.Select(options=ip_list,height=min(len(ip_list)*10,150),width=200)
    search_button = widgets.Button(description='Search')
    container.children = [ip_select,search_button]
    h_container.children = [label,container] 
    display(h_container)
    #display(container)
    
    def search_ip(b):
        global anchor_ip
        anchor_ip = ip_select.value       
        if anchor_ip != "":
            print "Searching for ip: " + anchor_ip
            global ir_f 
            ir_f = spath + "ir-" + anchor_ip.replace(".","_") + ".tsv"

            hivestr =  ("\"set mapred.max.split.size=1073741824;set hive.exec.reducers.max=10;set hive.cli.print.header=true;" + 
            "SELECT min(treceived) as firstSeen, max(treceived) as lastSeen, sip as srcIP, dip as dstIP, " + 
            "sport as SPort, dport AS Dport, count(sip) as conns, max(ipkt) as maxPkts, avg(ipkt) " + 
            "as avgPkts, max(ibyt) as maxBytes, avg(ibyt) as avgBytes FROM netflow WHERE " + 
            "year="+ t_date[0:4] +" AND month="+ t_date[4:6] +" AND day="+t_date[6:]+
            " AND (sip =\'" + anchor_ip + "\'  OR dip=\'" + anchor_ip + "\') GROUP BY sip, dip,sport,dport\"  > " + ir_f)
            print hivestr
            !hive -S -e $hivestr
    
    search_button.on_click(search_ip)

def details_inbound(ip, inbound):
    if ip != "":
        sbdet_f = spath + "sbdet-" + ip.replace(".","_") + ".tsv"
        hive_str = ("\"set mapred.max.split.size=1073741824;set hive.exec.reducers.max=10;set hive.cli.print.header=true;"
            + "SELECT min(treceived) as tstart, max(treceived) as tend, sip as srcIP, "
            + "dip as dstIP, proto as Proto, sport as SPort, dport AS Dport,ipkt as "
            + "Pkts, ibyt as Bytes  FROM netflow WHERE "
            +"year="+ t_date[0:4] +" AND month="+ t_date[4:6] +" AND day="+t_date[6:]
            + " AND (dip IN({0}) "
            + "AND sip ='{1}') OR "
            + "(sip IN({2}) "
            + "AND dip ='{3}') SORT BY tstart \"  > " + sbdet_f)

        ips = "'" + "','".join(inbound.keys()) + "'"
        hive_str = hive_str.format(ips,ip,ips,ip)

        print hive_str
        !hive -S -e $hive_str
                
def generate_attack_map_file(ip, inbound, outbound, twoway):
    # globe.html data files
    globe_fpath = spath + 'globe_' + anchor_ip.replace('.','_') + ".json"

    globe_json = {}

    globe_json['type'] = "FeatureCollection"
    globe_json['sourceips'] = []
    globe_json['destips'] = []

    for srcip in twoway:
        row =  twoway[srcip]['geo'].replace('"','').split(',')
        globe_json['destips'].append({
                'type': 'Feature',
                'properties': {
                    'location':row[8],
                    'ip':srcip,
                    'type':1
                },
                'geometry': {
                    'type': 'Point',
                    'coordinates': [float(row[7]), float(row[6])]
                }
            })

    for dstip in outbound:
        row =  outbound[dstip]['geo'].replace('"','').split(',')
        dst_geo = outbound[dstip]['geo_dst'].replace('"','').split(',')
        globe_json['sourceips'].append({
                'type': 'Feature',
                'properties': {
                    'location':row[8],
                    'ip':dstip,
                    'type':3
                },
                'geometry': {
                    'type': 'Point',
                    'coordinates': [float(row[7]), float(row[6])]
                }
            })
        globe_json['destips'].append({
                'type': 'Feature',
                'properties': {
                    'location':row[8],
                    'ip':outbound[dstip]['dst_ip'],
                    'type':3
                },
                'geometry': {
                    'type': 'Point',
                    'coordinates': [float(dst_geo[7]), float(dst_geo[6])]
                }
            })

    for dstip in inbound:
        row =  inbound[dstip]['geo'].replace('"','').split(',')
        dst_geo = inbound[dstip]['geo_src'].replace('"','').split(',')
        globe_json['sourceips'].append({
                'type': 'Feature',
                'properties': {
                    'location':row[8],
                    'ip':dstip,
                    'type':2
                },
                'geometry': {
                    'type': 'Point',
                    'coordinates': [float(row[7]), float(row[6])]
                }
            })
        globe_json['destips'].append({
                'type': 'Feature',
                'properties': {
                    'location':row[8],
                    'ip':inbound[dstip]['src_ip'],
                    'type':2
                },
                'geometry': {
                    'type': 'Point',
                    'coordinates': [float(dst_geo[7]), float(dst_geo[6])]
                }
            })

    json_str = json.dumps(globe_json)

    globe_f = open(globe_fpath, 'w')
    globe_f.write(json_str)
    globe_f.close()

def generate_dendro(ip, inbound, outbound, twoway, date):
    dendro_fpath = spath + 'dendro-' + anchor_ip.replace('.','_') + ".json"
    
    obj = {
        'name':ip,
        'children': [],
        'time': date        
    }
    
    #----- Add Inbound Connections-------#
    obj["children"].append({'name': 'Inbound Only', 'children': [], 'impact': 0})    
    in_ctxs = {}
    for ip in inbound:
        if 'nwloc' in inbound[ip] and len(inbound[ip]['nwloc']) > 0:
            ctx = inbound[ip]['nwloc'][2] # get the machine type Only for vast Data
            if ctx not in in_ctxs:
                in_ctxs[ctx] = 1
            else:
                in_ctxs[ctx] += 1
    for ctx in in_ctxs:
        obj["children"][0]['children'].append({
                'name': ctx,
                'impact': in_ctxs[ctx]
            })        
        
    
    #------ Add Outbound ----------------#
    obj["children"].append({'name': 'Outbound Only', 'children': [], 'impact': 0})
    out_ctxs = {}
    for ip in outbound:       
        if 'nwloc' in outbound[ip] and len(outbound[ip]['nwloc']) > 0:
            ctx = outbound[ip]['nwloc'][2] # get the machine type Only for vast Data
            if ctx not in out_ctxs:
                out_ctxs[ctx] = 1
            else:
                out_ctxs[ctx] += 1
    for ctx in out_ctxs:
        obj["children"][1]['children'].append({
                'name': ctx,
                'impact': out_ctxs[ctx]
            }) 
    
    #------ Add Outbound ----------------#
    obj["children"].append({'name': 'two way', 'children': [], 'impact': 0})
    tw_ctxs = {}
    for ip in twoway:
        if 'nwloc' in twoway[ip] and len(twoway[ip]['nwloc']) > 0:
            ctx = twoway[ip]['nwloc'][2] # get the machine type Only for vast Data
            if ctx not in tw_ctxs:
                tw_ctxs[ctx] = 1
            else:
                tw_ctxs[ctx] += 1
    
    for ctx in tw_ctxs:
        obj["children"][2]['children'].append({
                'name': ctx,
                'impact': tw_ctxs[ctx]
            })
    
    json_str = json.dumps(obj)

    dendro_f = open(dendro_fpath, 'w')
    dendro_f.write(json_str)
    dendro_f.close()

def generate_stats(ip, inbound, outbound, twoway, threat_name):
    stats_fpath = spath + 'stats-' + anchor_ip.replace('.','_') + ".json"
    
    obj = {
        'name':threat_name,
        'children': [],
        'size': len(inbound) + len(outbound) + len(twoway)
    }
    
    #----- Add Inbound Connections-------#
    obj["children"].append({'name': 'Inbound Only', 'children': [], 'size': len(inbound)})    
    in_ctxs = {}
    for ip in inbound:
        full_ctx = ''
        if 'nwloc' in inbound[ip] and len(inbound[ip]['nwloc']) > 0:
            full_ctx = inbound[ip]['nwloc'][2].split('.')[0]
        ctx = get_ctx_name(full_ctx) # get the machine type Only for vast Data
        if ctx not in in_ctxs:
            in_ctxs[ctx] = 1
        else:
            in_ctxs[ctx] += 1
    for ctx in in_ctxs:
        obj["children"][0]['children'].append({
                'name': ctx,
                'size': in_ctxs[ctx]
            })        
        
    
    #------ Add Outbound ----------------#
    obj["children"].append({'name': 'Outbound Only', 'children': [], 'size': len(outbound)})
    out_ctxs = {}
    for ip in outbound:
        full_ctx = ''
        if 'nwloc' in outbound[ip] and len(outbound[ip]['nwloc']) > 0:
            full_ctx = outbound[ip]['nwloc'][2].split('.')[0]
        ctx = get_ctx_name(full_ctx) # get the machine type Only for vast Data
        if ctx not in out_ctxs:
            out_ctxs[ctx] = 1
        else:
            out_ctxs[ctx] += 1
    for ctx in out_ctxs:
        obj["children"][1]['children'].append({
                'name': ctx,
                'size': out_ctxs[ctx]
            }) 
    
    #------ Add Twoway ----------------#
    obj["children"].append({'name': 'two way', 'children': [], 'size': len(twoway)})
    tw_ctxs = {}
    for ip in twoway:
        full_ctx = ''
        if 'nwloc' in twoway[ip] and len(twoway[ip]['nwloc']) > 0:
            full_ctx = twoway[ip]['nwloc'][2].split('.')[0]
        ctx = get_ctx_name(full_ctx) # get the machine type Only for vast Data
        if ctx not in tw_ctxs:
            tw_ctxs[ctx] = 1
        else:
            tw_ctxs[ctx] += 1
    
    for ctx in tw_ctxs:
        obj["children"][2]['children'].append({
                'name': ctx,
                'size': tw_ctxs[ctx]
            })
    
    json_str = json.dumps(obj)

    stats_f = open(stats_fpath, 'w')
    stats_f.write(json_str)
    stats_f.close()
    
def display_threat_box(ip):
    container_summary = widgets.HBox(width=550, height=150)
    separator = widgets.Box(width=550, height=15)       
    main_container = widgets.Box(width=550, height=200)       
    threat_title_box = widgets.Text(value='',width=200, placeholder='Threat Title')
    threat_summary_box = widgets.Textarea(value=''
                                      ,width=300, height=100)
    save_button = widgets.Button(description='Save')
    container_summary.children = [threat_summary_box,save_button]
    main_container.children = [threat_title_box, separator, container_summary] 
    display(main_container)
    
    def save_threat_summary(b):
        print spath
        file_name = spath + "threat_" + anchor_ip.replace(".", "_") + ".csv"
        file_content = "title,summary\n"
        file_content +=  threat_title_box.value + ',' + threat_summary_box.value.replace("\n", "<br/>") + "\n"
        global threat_name
        threat_name = threat_title_box.value
        threat_f = open(file_name, 'w')
        threat_f.write(file_content)
        threat_f.close()
        add_threat(anchor_ip, threat_title_box.value)
        print "Saved file {0}".format(file_name)
    
    save_button.on_click(save_threat_summary)
    
def get_ctx_name(full_context):    
    ctx= 'DMZ'
    if "VPN" in full_context:
        ctx = "VPN" 
    elif "DMZ" in full_context:
        ctx = "DMZ"
    elif "Proxy" in full_context:
        ctx = "Proxy" 
    elif "FW" in full_context:
        ctx = "FW"     
    return ctx

def display_expanded_search():
    external_ips = []
    with open(sconnect, 'r') as f:
        reader = csv.reader(f, delimiter=',')
        reader.next()
        for row in reader:
            #sev,tstart,srcIP,dstIP,sport,dport,proto,ipkt,ibyt,lda_score,srcIpInternal,destIpInternal,srcGeo,dstGeo,srcDomain,dstDomain
            # 0 ,  1   ,  2  ,  3  ,  4  ,  5  ,  6  ,  7 ,  8 ,   9     ,  10         ,    11        ,  12  ,  13  ,  14     ,  13
            #if row[0] == '1':
            srcIP = ''
            dstIP = ''
            if row[10] == '0' and row[2] not in external_ips:
                external_ips.append(row[2])
            if row[11] == '0' and row[3] not in external_ips:
                external_ips.append(row[3])

    if len(external_ips) > 0:
        display_controls(external_ips)
    else:
        print "There are not high risk connections."

# calculate number of inbound only, two-way, and outbound only
# build dict of IP addresses
# firstSeen,lastSeen,srcIP, dstIP, sport,dport,conns, maxPkts, avgPkts,maxBytes, avgBytes
def get_in_out_and_twoway_conns():
    global inbound
    inbound = {}
    global outbound
    outbound = {}
    global twoway
    twoway = {}
    srcdict = {}
    dstdict = {}
    conns_dict= {}
    #ir_f = spath + 'ir-130_133_4_11.tsv'
    with open(ir_f, 'r') as f:
        reader = csv.reader(f,delimiter='\t') 
        reader.next();
        rowct = 1
        for row in reader:
            srcdict[row[2]] = {
                'ip_int': struct.unpack("!L", socket.inet_aton(row[2]))[0],
                'dst_ip': row[3],
                'dst_ip_int': struct.unpack("!L", socket.inet_aton(row[3]))[0],
                'conns': int(row[6]),
                'maxbytes': int(row[9])
            }
            dstdict[row[3]] = {
                 'ip_int': struct.unpack("!L", socket.inet_aton(row[3]))[0],
                 'src_ip': row[2],
                 'src_ip_int': struct.unpack("!L", socket.inet_aton(row[2]))[0],
                 'conns': int(row[6]),
                 'maxbytes': int(row[9])
                }                   
            rowct += 1
            
    for result in srcdict:
        if result in dstdict:
            twoway[result] = srcdict[result]
        else:
            outbound[result] = srcdict[result]

    for result in dstdict:
        if result not in srcdict:
            inbound[result] = dstdict[result]  

#=========== Adds GEO IP information to the outbound, inbound and twoway connections==============================#
def add_geospatial_info():
    # get geospatial info
    if len(outbound.keys()) > 0:
        print "outbound connections located."
    for srcip in outbound:
        outbound[srcip]['geo'] = linecache.getline(iploc, bisect.bisect(iplist,outbound[srcip]['ip_int']))
        outbound[srcip]['geo_dst'] = linecache.getline(iploc, bisect.bisect(iplist,outbound[srcip]['dst_ip_int']))
    
    if len(twoway.keys()) > 0:
        print "twoway connections located."
    for dstip in twoway:
        twoway[dstip]['geo'] = linecache.getline(iploc, bisect.bisect(iplist,twoway[dstip]['ip_int']))
    
    if len(inbound.keys()) > 0:
        print "inbound connections located."
    for srcip in inbound:
        inbound[srcip]['geo'] = linecache.getline(iploc, bisect.bisect(iplist,inbound[srcip]['ip_int']))
        inbound[srcip]['geo_src'] = linecache.getline(iploc, bisect.bisect(iplist,inbound[srcip]['src_ip_int']))
        
# need some way to combine timelines of outbound and two-way with big picture inbound only
# get network context - get start and end ranges
def add_network_context():
    nwdict = {}
    with open(nwloc, 'r') as f:
        reader = csv.reader(f,delimiter=',') 
        #reader.next()   
        rowct = 1
        #address range, description
        for row in reader:
            iprange = row[0].split('/')        
            if len(iprange) < 2:
                ipend = 0
            else:
                ipend = int(iprange[1])

            nwdict[row[0]] = [struct.unpack("!L", socket.inet_aton(iprange[0]))[0],
                              struct.unpack("!L", socket.inet_aton(iprange[0]))[0]+2**(32-ipend)-1,
                              row[1]]       
            rowct += 1
        
    if len(outbound.keys()) > 0:
        print "outbound connections located."        
    for srcip in outbound:        
        temp_ip = srcip.split('.')[0] + '.0.0.0'           
        outbound[srcip]['nwloc'] = nwdict[temp_ip] if temp_ip in nwdict else  ''
    
    if len(twoway.keys()) > 0:
        print "twoway connections located."
    for dstip in twoway:        
        temp_ip = dstip.split('.')[0] + '.0.0.0'        
        twoway[dstip]['nwloc'] = nwdict[temp_ip] if temp_ip in nwdict else  ''
    
    if len(inbound.keys()) > 0:
        print "inbound connections located."
    for srcip in inbound:
        temp_ip = srcip.split('.')[0] + '.0.0.0'           
        inbound[srcip]['nwloc'] = nwdict[temp_ip] if temp_ip in nwdict else  ''

def add_threat(ip,threat_title):
    content = ''
    try:
        threat_f = open(threats_file, 'r')
        content = threat_f.read()
        if '{0},{1}\n'.format(ip,threat_title) not in content:
            content += '{0},{1}\n'.format(ip,threat_title)
        threat_f.close()
    except:
        content = 'ip,threatTitle\n'
        content += '{0},{1}\n'.format(ip,threat_title)
    
    threat_fw = open(threats_file, 'w')
    threat_fw.write(content)
    threat_fw.close()
    
def get_top_bytes(conns_dict, top):
    print "Now looking at the top "+ str(top) +" connections per bytes:"
    topbytes = sorted(conns_dict.iteritems(), key=lambda (x,y): y['maxbytes'], reverse=True)
    topbytes = topbytes[0:top]
    for item in topbytes:
        print item[0], "|", item[1]["maxbytes"]
    return dict(topbytes)

def get_top_conns(conns_dict, top):
    print "Now looking at the top "+str(top)+" connections per number of connections:"
    topconns = sorted(conns_dict.iteritems(), key=lambda (x,y): y['conns'], reverse=True)
    topconns = topconns[0:50]
    for item in topconns:
        print item[0], "|", item[1]["conns"]
    return dict(topconns)